In [1]:
import pandas as pd
import sys
import numpy as np
import cv2

sys.path.append('..')

from airbus_ship_detection import utility

In [2]:
df = pd.read_csv('../input/train_ship_segmentations_v2.csv')
df.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [3]:
# Create column that has count of ships in image
image_ship_counts = df.groupby('ImageId').size().reset_index(name='count')

# Set count of images with no ship to 0
images_with_no_ships = df.loc[df.EncodedPixels.isna()].ImageId.values
image_ship_counts.loc[df['ImageId'].isin(images_with_no_ships), 'count'] = 0

image_ship_counts.head()

,ImageId,count
0,00003e153.jpg,0
1,0001124c7.jpg,0
2,000155de5.jpg,1
3,000194a2d.jpg,5
4,0001b1832.jpg,1


In [4]:
pd.merge(image_ship_counts, df)

,ImageId,count,EncodedPixels
0,00003e153.jpg,0,NaN
1,0001124c7.jpg,0,NaN
2,000155de5.jpg,1,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,5,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,5,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...
...,...,...,...
231718,fffedbb6b.jpg,0,NaN
231719,ffff2aa57.jpg,0,NaN
231720,ffff6e525.jpg,1,NaN
231721,ffffc50b4.jpg,0,NaN


In [5]:
from IPython.core.debugger import set_trace

In [8]:
from torch.utils.data import Dataset
from skimage.io import imread
from torchvision.transforms import ToTensor, Compose, Normalize

class ShipDataset(Dataset):
    
    def __init__(self, df):
        self.image_ids = df.ImageId.unique()
        grp = list(df.groupby('ImageId'))
        set_trace()
        self.image_masks = [m['EncodedPixels'].values for _,m in grp]
        self.img_transform = Compose([
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        file_name = self.image_ids[idx]
        image = imread(f'../input/train_v2/{file_name}')
        return self.img_transform(image)

In [ ]:
ds = ShipDataset(image_ship_counts[:10])

> <ipython-input-8-0f121872aeb4>(11)__init__()
      9         grp = list(df.groupby('ImageId'))
     10         set_trace()
---> 11         self.image_masks = [m['EncodedPixels'].values for _,m in grp]
     12         self.img_transform = Compose([
     13             ToTensor(),



ipdb>  grp[5][1]


         ImageId  count
5  00021ddc3.jpg      9


ipdb>  grp[5][1].ImageId


5    00021ddc3.jpg
Name: ImageId, dtype: object


In [ ]:
ds[15].shape

In [ ]:
from skimage.io import imshow
imshow(ds[15])

In [ ]:
filename = image_ship_counts.loc[15].ImageId
total_mask = np.zeros((768, 768))
for mask in df.loc[df.ImageId == filename, 'EncodedPixels'].values:
    mask_img = utility.rle_decode(mask, shape=(768,768))
    total_mask += mask_img
    imshow(total_mask)

In [ ]:
m = np.dstack((total_mask, total_mask, total_mask)) * np.array((0, 1, 0))
imshow(cv2.addWeighted(ds[15], 0.9, m.astype('uint8')*128, 0.9, 0))

In [ ]:
grp = list(df.groupby('ImageId'))

In [ ]:
grp[15][1].EncodedPixels.values

In [ ]:
def masks_as_image(mask_list):
    all_masks = np.zeros((768, 768), dtype=np.int16)
    for mask in mask_list:
        if isinstance(mask, str):
            all_masks += utility.rle_decode(mask, (768, 768))
    return np.expand_dims(all_masks, -1)

mask = masks_as_image(grp[15][1].EncodedPixels.values)
print(mask.shape)

In [ ]:
import torch
torch.from_numpy(np.moveaxis(mask, -1, 0)).float().shape

In [ ]:
# Implementation from https://github.com/timctho/unet-pytorch/
class UNet_down_block(torch.nn.Module):
    def __init__(self, input_channel, output_channel, down_size):
        super(UNet_down_block, self).__init__()
        self.conv1 = torch.nn.Conv2d(input_channel, output_channel, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(output_channel)
        self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(output_channel)
        self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(output_channel)
        self.max_pool = torch.nn.MaxPool2d(2, 2)
        self.relu = torch.nn.ReLU()
        self.down_size = down_size

    def forward(self, x):
        if self.down_size:
            x = self.max_pool(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        return x

class UNet_up_block(torch.nn.Module):
    def __init__(self, prev_channel, input_channel, output_channel):
        super(UNet_up_block, self).__init__()
        self.up_sampling = torch.nn.Upsample(scale_factor=2, mode='bilinear')
        self.conv1 = torch.nn.Conv2d(prev_channel + input_channel, output_channel, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(output_channel)
        self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(output_channel)
        self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(output_channel)
        self.relu = torch.nn.ReLU()

    def forward(self, prev_feature_map, x):
        x = self.up_sampling(x)
        x = torch.cat((x, prev_feature_map), dim=1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        return x


class UNet(torch.nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.down_block1 = UNet_down_block(3, 16, False)
        self.down_block2 = UNet_down_block(16, 32, True)
        self.down_block3 = UNet_down_block(32, 64, True)
        self.down_block4 = UNet_down_block(64, 128, True)
        self.down_block5 = UNet_down_block(128, 256, True)
        self.down_block6 = UNet_down_block(256, 512, True)
        self.down_block7 = UNet_down_block(512, 1024, True)

        self.mid_conv1 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(1024)
        self.mid_conv2 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(1024)
        self.mid_conv3 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(1024)

        self.up_block1 = UNet_up_block(512, 1024, 512)
        self.up_block2 = UNet_up_block(256, 512, 256)
        self.up_block3 = UNet_up_block(128, 256, 128)
        self.up_block4 = UNet_up_block(64, 128, 64)
        self.up_block5 = UNet_up_block(32, 64, 32)
        self.up_block6 = UNet_up_block(16, 32, 16)

        self.last_conv1 = torch.nn.Conv2d(16, 16, 3, padding=1)
        self.last_bn = torch.nn.BatchNorm2d(16)
        self.last_conv2 = torch.nn.Conv2d(16, 1, 1, padding=0)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        self.x1 = self.down_block1(x)
        self.x2 = self.down_block2(self.x1)
        self.x3 = self.down_block3(self.x2)
        self.x4 = self.down_block4(self.x3)
        self.x5 = self.down_block5(self.x4)
        self.x6 = self.down_block6(self.x5)
        self.x7 = self.down_block7(self.x6)
        self.x7 = self.relu(self.bn1(self.mid_conv1(self.x7)))
        self.x7 = self.relu(self.bn2(self.mid_conv2(self.x7)))
        self.x7 = self.relu(self.bn3(self.mid_conv3(self.x7)))
        x = self.up_block1(self.x6, self.x7)
        x = self.up_block2(self.x5, x)
        x = self.up_block3(self.x4, x)
        x = self.up_block4(self.x3, x)
        x = self.up_block5(self.x2, x)
        x = self.up_block6(self.x1, x)
        x = self.relu(self.last_bn(self.last_conv1(x)))
        x = self.last_conv2(x)
        return x

In [ ]:
model = UNet()

In [ ]:
from torchsummary import summary

summary(model, input_size=(3, 768, 768))

In [ ]:
 class LossBinary:
    """
     Implementation from  https://github.com/ternaus/robot-surgery-segmentation
    """

    def __init__(self, jaccard_weight=0):
        self.nll_loss = torch.nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        loss = self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            eps = 1e-15
            jaccard_target = (targets == 1.0).float()
            jaccard_output = F.sigmoid(outputs)

            intersection = (jaccard_output * jaccard_target).sum()
            union = jaccard_output.sum() + jaccard_target.sum()

            loss -= self.jaccard_weight * torch.log((intersection + eps) / (union - intersection + eps))
        return loss



In [ ]:
criterion = LossBinary(5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset=ds, batch_size=32)

In [ ]:
for epoch in range(2):
    for i, (inputs, targets) in enumerate(loader):
        ouputs = model(inputs)